# 🦟 Dengue Risk Prediction Notebook
This notebook demonstrates a simple **rule-based** dengue danger scoring system.

**Inputs requested:**
1. Area name (e.g., Ang Mo Kio, Bukit Panjang, Toa Payoh)
2. Presence of construction sites (yes/no)
3. Humidity (%)
4. Temperature (°C)
5. Average rainfall last 7 days (mm)
6. Historical dengue cases in that area (last week)

**Outputs:**
- Danger level (High/Medium/Low) with colour identifier
- A dummy 4‑week timeline chart of risk score
- Mitigation suggestions


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings, os
warnings.filterwarnings('ignore')

In [ ]:
### 📝 Collect User Inputs and Auto-fetch (when possible)
area = input('Enter area name (e.g., AMK, BP): ').strip()
# OPENWEATHER_API_KEY can be provided as an environment variable named OPENWEATHER_API_KEY
OWM_API_KEY = os.environ.get('OPENWEATHER_API_KEY')

def fetch_weather_for_area(area, api_key=OWM_API_KEY):
    import requests
    if not api_key or not area:
        return None
    try:
        # 1) Geocode the area name (OpenWeather Geocoding API)
        geo_url = f'http://api.openweathermap.org/geo/1.0/direct?q={area}&limit=1&appid={api_key}'
        r = requests.get(geo_url, timeout=10)
        r.raise_for_status()
        geo = r.json()
        if not geo:
            return None
        lat, lon = geo[0]['lat'], geo[0]['lon']
        # 2) Fetch current weather (One Call) — current only to keep within free-tier usage
        weather_url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=minutely,hourly,daily,alerts&units=metric&appid={api_key}'
        r2 = requests.get(weather_url, timeout=10)
        r2.raise_for_status()
        w = r2.json().get('current', {})
        humidity = w.get('humidity')
        temperature = w.get('temp')
        # rainfall may be present under 'rain' -> '1h' for the last hour; we treat missing as 0
        rainfall = 0.0
        if 'rain' in w:
            rainfall = float(w.get('rain', {}).get('1h', 0.0))
        return {'humidity': humidity, 'temperature': temperature, 'rainfall': rainfall}
    except Exception:
        return None

# Try to auto-fetch weather data for the entered area
weather = fetch_weather_for_area(area)
if weather is None:
    print('Auto-fetch not available (no API key or request failed). You will be prompted for values.')
    construction = input('Construction sites present? (yes/no): ').strip().lower() == 'yes'
    humidity = float(input('Humidity level (%): '))
    temperature = float(input('Temperature (°C): '))
    rainfall = float(input('Average rainfall last 7 days (mm): '))
code
#VSC-fdab8146
python
colour_map = {'High':'red', 'Medium':'yellow', 'Low':'green'}
# Use computed `danger` from scoring cell
try:
    disp_danger = danger
except NameError:
    disp_danger = 'Low'

# Draw a circle
fig, ax = plt.subplots()
circle = plt.Circle((0.5, 0.5), 0.3, color=colour_map.get(disp_danger, 'green'))
ax.add_patch(circle)

# Hide axes
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

plt.title(f"Dengue Danger Level: {disp_danger}", fontsize=14)
plt.show()

print('Colour identifier:', colour_map.get(disp_danger, 'green'))
    except Exception:
        historical_cases = None

if historical_cases is None:
    try:
        historical_cases = int(input('Dengue cases last week (enter 0 if unknown): '))
    except Exception:
        historical_cases = 0

print(f'Inputs: area={area}, construction={construction}, humidity={humidity}, temperature={temperature}, rainfall={rainfall}, historical_cases={historical_cases}')

In [ ]:
### 🌐 Fetch Real-Time Dengue Case Data
# This cell attempts to fetch official dengue case counts from public health APIs
# Fallback: manual entry or local CSV if API unavailable

def fetch_dengue_cases_realtime(area):
    """
    Attempt to fetch real-time dengue cases for the area from available APIs.
    Sources tried in order:
    1. Singapore MOH / NEA data (if available)
    2. OpenDengue / Dengue mapping services
    3. Local CSV fallback
    Returns: (cases_count, clusters_count, data_source) or (None, None, 'manual')
    """
    import requests
    
    # Try 1: Singapore government dengue data (example endpoint - adjust based on actual API)
    # Note: Replace with actual Singapore MOH API if available
    try:
        # Example using a hypothetical dengue API endpoint
        url = f'https://api.dengueforecasting.org/v1/dengue/current?location={area}&country=SG'
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        if data.get('cases'):
            return int(data['cases']), int(data.get('clusters', 0)), 'OpenDengue API'
    except:
        pass
    
    # Try 2: Local CSV with area-based lookup
    try:
        import pandas as pd
        if os.path.exists('dengue_cases.csv'):
            df = pd.read_csv('dengue_cases.csv')
            # Case-insensitive match
            area_lower = area.lower()
            match = df[df.iloc[:, 0].str.lower() == area_lower]
            if not match.empty:
                cases = int(match.iloc[0, 1])
                clusters = int(match.iloc[0, 2]) if len(match.columns) > 2 else 0
                return cases, clusters, 'Local CSV'
    except:
        pass
    
    return None, None, 'manual'

# Attempt real-time fetch
print('Fetching real-time dengue case data for', area, '...')
rt_cases, rt_clusters, source = fetch_dengue_cases_realtime(area)

if rt_cases is not None:
    print(f'[{source}] Cases: {rt_cases}, Clusters: {rt_clusters}')
    historical_cases = rt_cases
    active_clusters = rt_clusters
else:
    print('Real-time data unavailable. Manual entry required.')
    try:
        historical_cases = int(input('Dengue cases last week (or 0 if unknown): '))
        active_clusters = int(input('Active clusters (or 0 if unknown): '))
    except:
        historical_cases = 0
        active_clusters = 0

print(f'Using: {historical_cases} cases, {active_clusters} active clusters')


In [ ]:
### 🔢 Danger Scoring (improved heuristic)
# Weighted scoring with simple continuous adjustments
score = 0.0
# construction sites increase breeding opportunities
score += 2 if construction else 0
# humidity contribution (scaled)
if humidity is not None:
    # baseline 60%; scale above baseline up to +2
    score += max(0, min(2, (float(humidity) - 60) / 7.5))
# temperature contribution (ideal range 25-32 -> higher risk near 30)
if temperature is not None:
    t = float(temperature)
    if t >= 30:
        score += 1.5
    elif t >= 26:
        score += 0.8
# rainfall: heavier recent rain increases breeding sites (scaled)
if rainfall is not None:
    r = float(rainfall)
    if r >= 50:
        score += 2
    elif r >= 20:
        score += 1
# historical cases: strongest signal
if historical_cases >= 20:
    score += 3
elif historical_cases >= 5:
    score += 1

# Normalize score to integer 0-10 range for readability
score = int(min(10, round(score)))
if score >= 6:
    danger = 'High'
elif score >= 3:
    danger = 'Medium'
else:
    danger = 'Low'

# Action level mapping
action_map = {
    'High': 'Immediate — Enhanced surveillance, fogging where appropriate, community alerts, and intensive source reduction',
    'Medium': 'Targeted inspections, community outreach, larvicide in hotspots, weekly monitoring',
    'Low': 'Routine monitoring and public education; update weekly'
}
action = action_map[danger]

# Tailored mitigation suggestions based on drivers
suggestions = []
if construction:
    suggestions.append('Inspect construction sites and drains; enforce water management on sites')
if humidity >= 75:
    suggestions.append('Increase community checks for stagnant water as humid conditions favour mosquitoes')
if temperature >= 30:
    suggestions.append('Focus on outdoor breeding spots; adult mosquitoes more active')
if rainfall >= 50:
    suggestions.append('Clear debris and containers that collect rainwater; check drains')
if historical_cases >= 5:
    suggestions.append('Conduct targeted house-to-house inspections and public awareness in affected clusters')

# Fallback generic suggestions
if not suggestions:
    suggestions = [
,
,

Danger level for Bukit Panjang: High (score 6)


In [ ]:
### 💾 Save Result to CSV Report
import pandas as pd
from datetime import datetime

# Prepare row for report
report_row = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'area': area,
    'construction': construction,
    'humidity_pct': humidity,
    'temperature_c': temperature,
    'rainfall_mm': rainfall,
    'historical_cases': historical_cases,
    'danger_score': score,
    'danger_level': danger,
    'recommended_action': action,
    'suggestions': ' | '.join(suggestions)
}

report_file = 'dengue_report.csv'
# Append to CSV (create if not exists)
if os.path.exists(report_file):
    df_existing = pd.read_csv(report_file)
    df_new = pd.DataFrame([report_row])
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined.to_csv(report_file, index=False)
    print(f'✓ Appended result to {report_file}')
else:
    df_new = pd.DataFrame([report_row])
    df_new.to_csv(report_file, index=False)
    print(f'✓ Created new {report_file}')

print(f'\nReport row saved:')
print(report_row)
